In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/ten

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))) / predictions.shape[0]

## Problem 1

In [11]:
batch_size = 128
n_hidden = 1024
lamda = 0.004

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## Hidden Layer
  hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  hidden_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_result = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  

  ## Output Layer
  output_weights = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_result, output_weights) + output_biases

  error = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  
  # Training computation.
  loss = tf.reduce_mean(error) + lamda * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1722.153931
Minibatch accuracy: 6.2%
Validation accuracy: 36.0%
Minibatch loss at step 500: 168.253098
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 22.961842
Minibatch accuracy: 84.4%
Validation accuracy: 85.4%
Minibatch loss at step 1500: 3.805091
Minibatch accuracy: 81.2%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 1.098708
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 0.574248
Minibatch accuracy: 90.6%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 0.565269
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Test accuracy: 91.4%


In [27]:
from sklearn.linear_model import LogisticRegression

train_subset = 10000
lg_train_labels =  np.argmax(train_labels, 1)
lg_test_labels =  np.argmax(test_labels, 1)
clf = LogisticRegression(random_state = 42, multi_class='multinomial', solver='lbfgs', penalty = 'l2')
clf.fit(train_dataset[:train_subset], lg_train_labels[:train_subset])
clf.score(test_dataset, lg_test_labels)

/home/mike/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8534

## Problem 2

In [5]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## Hidden Layer
  hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  hidden_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_result = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  

  ## Output Layer
  output_weights = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_result, output_weights) + output_biases

  error = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  
  # Training computation.
  loss = tf.reduce_mean(error)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 3001
train_dataset_reduced = train_dataset[:580, :]
train_labels_reduced = train_labels[:580]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels_reduced.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset_reduced[offset:(offset + batch_size), :]
    batch_labels = train_labels_reduced[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 417.836304
Minibatch accuracy: 10.2%
Validation accuracy: 29.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Minibatch loss at step 1000: 0.063590
Minibatch accuracy: 99.2%
Validation accuracy: 73.1%
Minibatch loss at step 1500: 0.000816
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Minibatch loss at step 2000: 0.003439
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Minibatch loss at step 2500: 0.000797
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 0.001724
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Test accuracy: 81.7%


#### From the above result, we know too short data occurs overfitting

## Problem 3

In [11]:
batch_size = 128
n_hidden = 1024
keep_prob = 0.8

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## Dropout  
  dropout = tf.placeholder(tf.float32)
  tf_train_dataset_dropout = tf.nn.dropout(tf_train_dataset, dropout)
    
  ## Hidden Layer
  hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  hidden_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_result = tf.nn.relu(tf.matmul(tf_train_dataset_dropout, hidden_weights) + hidden_biases)
  

  ## Output Layer
  output_weights = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_result, output_weights) + output_biases

  error = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  
  # Training computation.
  loss = tf.reduce_mean(error)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, output_weights) + output_biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout: keep_prob}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 360.319580
Minibatch accuracy: 7.0%
Validation accuracy: 34.0%
Minibatch loss at step 500: 34.921852
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 21.613770
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 16.105671
Minibatch accuracy: 75.8%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 18.757210
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 20.695438
Minibatch accuracy: 73.4%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 11.011712
Minibatch accuracy: 80.5%
Validation accuracy: 80.7%
Test accuracy: 87.7%


## Problem 4

In [12]:
batch_size = 128
n_hidden = 1024
n_hidden2 = 800
learning_rate = 0.05
num_steps = 3001
keep_prob = 0.9
decay_rate = 0.9
decay_steps = 100
lamda = 0.001

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  ## learning rate decay
#   global_step = tf.Variable(0)
#   learning_rate_decay = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate)

  ## Dropout  
  dropout = tf.placeholder(tf.float32)
  tf_train_dataset_dropout = tf.nn.dropout(tf_train_dataset, dropout)
    
  ## Hidden Layer
  hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden],stddev = np.sqrt(2.0/n_hidden)))
  hidden_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_result = tf.nn.relu(tf.matmul(tf_train_dataset_dropout, hidden_weights) + hidden_biases)
    
  hidden_result_dropout = tf.nn.dropout(hidden_result, dropout)  
    
  ## Second Hidden Layer
  hidden2_weights = tf.Variable(tf.truncated_normal([n_hidden, n_hidden2],stddev = np.sqrt(2.0/n_hidden2)))
  hidden2_biases = tf.Variable(tf.zeros([n_hidden2]))
  hidden2_result = tf.nn.relu(tf.matmul(hidden_result_dropout, hidden2_weights) + hidden2_biases)  
  

  ## Output Layer
  output_weights = tf.Variable(tf.truncated_normal([n_hidden2, num_labels],stddev = np.sqrt(2.0/num_labels)))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden2_result, output_weights) + output_biases

  error = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  
  # Training computation.
  loss = tf.reduce_mean(error) + lamda * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden2_weights) 
                                          + tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden, hidden2_weights) + hidden2_biases)  
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden2, output_weights) + output_biases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden, hidden2_weights) + hidden2_biases) 
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden2, output_weights) + output_biases)

In [13]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  global_ = tf.Variable(tf.constant(0))
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout:keep_prob}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.241568
Minibatch accuracy: 12.5%
Validation accuracy: 33.5%
Minibatch loss at step 500: 2.435693
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 2.300859
Minibatch accuracy: 83.6%
Validation accuracy: 86.3%
Minibatch loss at step 1500: 2.247794
Minibatch accuracy: 84.4%
Validation accuracy: 87.5%
Minibatch loss at step 2000: 2.113325
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 2500: 1.811457
Minibatch accuracy: 92.2%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 1.802435
Minibatch accuracy: 91.4%
Validation accuracy: 88.2%
Test accuracy: 94.6%
